# Different ways to access [OpenStreetMap](https://www.openstreetmap.org)

- [Foundations of Python Network Programming Ch1](https://github.com/brandon-rhodes/fopnp/tree/m/py3/chapter01)

## 1. Using geopy

In [2]:
# use  openstreetmap.org
# prerequisite: pip3 install geopy
from geopy.geocoders import Nominatim

if __name__ == '__main__':
    address = '4700 Research Way, Lakeland, FL 33805'
    user_agent = 'Find the latitude and longitute of a place using geopy'
    location = Nominatim(user_agent=user_agent).geocode(address)
    print(location.latitude, location.longitude)

28.147705350000003 -81.84878516048089


## 2. Using HTTP requests

In [3]:
import requests

def geocode(address):
    base = 'https://nominatim.openstreetmap.org/search'
    parameters = {'q': address, 'format': 'json'}
    user_agent = 'Find the latitude and longitute of a place using HTTP requests'
    headers = {'User-Agent': user_agent}
    response = requests.get(base, params=parameters, headers=headers)
    reply = response.json()
    print(reply[0]['lat'], reply[0]['lon'])

address = '4700 Research Way, Lakeland, FL 33805'
geocode(address)

28.142033 -81.86029


## 3. Using HTTP

In [5]:
import http.client
import json
from urllib.parse import quote_plus

base = '/search'

def geocode(address):
    path = '{}?q={}&format=json'.format(base, quote_plus(address))
    user_agent = b'Find the latitude and longitute of a place using HTTP client'
    headers = {b'User-Agent': user_agent}
    connection = http.client.HTTPSConnection('nominatim.openstreetmap.org')
    connection.request('GET', path, None, headers)
    rawreply = connection.getresponse().read()
    reply = json.loads(rawreply.decode('utf-8'))
    print(reply[0]['lat'], reply[0]['lon'])

address = '4700 Research Way, Lakeland, FL 33805'
geocode(address)

28.142033 -81.86029


## 4. Using socket

In [31]:
import socket
import ssl,certifi
from urllib.parse import quote_plus
import os
import json

request_text = """\
GET /search?q={}&format=json HTTP/1.1\r\n\
Host: nominatim.openstreetmap.org\r\n\
User-Agent: Find the latitude and longitute of a place using socket\r\n\
Connection: close\r\n\
\r\n\
"""

def geocode(address):
    ci = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT);
    ci.verify_mode = ssl.CERT_REQUIRED;
    ci.load_verify_locations(cafile=os.path.relpath(certifi.where()), capath=None, cadata=None)
    unencrypted_sock = socket.socket()
    sock = ci.wrap_socket(unencrypted_sock, server_hostname='nominatim.openstreetmap.org')
    sock.connect(('nominatim.openstreetmap.org', 443))
    request = request_text.format(quote_plus(address))
    sock.sendall(request.encode('utf-8'))
    raw_reply = b''
    while True:
        more = sock.recv(4096)
        if not more:
            break
        raw_reply += more
    reply = raw_reply.decode('utf-8')
    print(reply)


address = '4700 Research Way, Lakeland, FL 33805'
geocode(address)

HTTP/1.1 200 OK
Server: nginx
Date: Tue, 05 Sep 2023 18:24:53 GMT
Content-Type: application/json; charset=utf-8
Content-Length: 4264
Connection: close

[{"place_id":359194069,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright","osm_type":"way","osm_id":279425009,"lat":"28.142033","lon":"-81.86029","class":"place","type":"house","place_rank":30,"importance":9.999999994736442e-08,"addresstype":"place","name":"","display_name":"4700, Research Way, Lakeland, Polk County, Florida, 33805, United States","boundingbox":["28.1419830","28.1420830","-81.8603400","-81.8602400"]},{"place_id":293593932,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright","osm_type":"way","osm_id":279425023,"lat":"28.147705350000003","lon":"-81.84878516048089","class":"amenity","type":"university","place_rank":30,"importance":9.99999999995449e-06,"addresstype":"amenity","name":"Florida Polytechnic University","display_name":"Florida Polytechnic University, 47